In [ ]:
# import libraries

from scipy.stats import sem as sem
import sys
import os
import glob
import BioSimSpace as BSS

if '/home/anna/Documents/cinnabar' not in sys.path:
    sys.path.insert(1, '/home/anna/Documents/cinnabar')
import cinnabar

print("adding code to the pythonpath...")
code = '/home/anna/Documents/code/python'
if code not in sys.path:
    sys.path.insert(1, code)
import pipeline

print(cinnabar.__file__)

from pipeline import *
from pipeline.utils import *
from pipeline.analysis import *

In [ ]:
bench_folder = f"/home/anna/Documents/benchmark"
protein = "tyk2"
main_dir = f"{bench_folder}/extracted/{protein}"

# choose location for the files
net_file = f"{main_dir}/execution_model/network_lomap.dat"
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
exp_file = f"{bench_folder}/inputs/experimental/{protein}.yml"

if os.path.exists(f"{main_dir}/outputs_extracted/results"):
    results_folder = f"{main_dir}/outputs_extracted/results"
elif os.path.exists(f"{main_dir}/outputs/results"):
    results_folder = f"{main_dir}/outputs/results"
# else:
#     raise ValueError(f"results directory not found in the {main_dir}. please make sure results were written using the analysis script previously in the pipeline")

output_folder = validate.folder_path(f"{main_dir}/analysis/comparing_methods", create=True)


In [ ]:
engine = "AMBER"
all_analysis_object = analysis_network(results_folder,
                                       exp_file=exp_file,
                                       net_file=net_file,
                                       engines=engine,
                                       output_folder=output_folder,
                                       analysis_ext=ana_file,
                                       # name="softcore",
                                        )

# can add any other results files
# all_analysis_object.compute_other_results(file_name=None, name=None)
all_analysis_object.compute(cycle_closure=False, use_cinnabar=True)

In [ ]:
write_modified_results_files(f"{results_folder}/final_summary_GROMACS_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv", name="switch", engine="GROMACS")
write_modified_results_files(f"{results_folder}/final_summary_GROMACS_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv", name="v_rescale", engine="GROMACS")

In [ ]:
all_analysis_object.compute_other_results(glob.glob(f'{results_folder}/final_summary_{engine}_MBAR_alchemlyb_None_eqfalse_statstrue_truncate0end.csv'), name="stats", use_cinnabar=True)
all_analysis_object.compute_other_results(glob.glob(f'{results_folder}/final_summary_{engine}_MBAR_alchemlyb_None_eqtrue_statsfalse_truncate0end.csv'), name="eq", use_cinnabar=True)
all_analysis_object.compute_other_results(glob.glob(f'{results_folder}/final_summary_{engine}_MBAR_alchemlyb_None_eqtrue_statstrue_truncate0end.csv'), name="both", use_cinnabar=True)
# all_analysis_object.compute_other_results(f"{results_folder}/results_0_GROMACS_v_rescale.csv", name="v_rescale", use_cinnabar=True)


In [ ]:
print(all_analysis_object.successful_runs(eng=engine))
# all_analysis_object.draw_graph()
all_analysis_object.remove_perturbations(all_analysis_object.failed_runs(eng=engine))  
all_analysis_object.remove_ligands(all_analysis_object.disconnected_ligands(eng=engine))
all_analysis_object.compute(cycle_closure=True, use_cinnabar=True)
all_analysis_object.draw_graph()

In [ ]:
all_analysis_object._plotting_object.bar(pert_val="pert", names=[engine,"stats","eq","both","experimental"]) # , values=["lig_35~lig_49", "lig_60~lig_61", "lig_56~lig_59"]

In [ ]:
for meth in ["stats","eq","both",engine]:
    print(f"{meth} : {all_analysis_object._stats_object.compute_mue('val', y=meth)}")
